In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的E2E ML：MLOps阶段2：使用客户管理的加密密钥（CMEK）进行AutoML图像分类训练

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_with_cmek_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_with_cmek_training.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab上运行
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_with_cmek_training.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

##概述

本教程演示了如何在谷歌云中使用顶点 AI 进行端对端 MLOps 生产过程。本教程涵盖了第二阶段：实验：使用客户管理加密密钥 CMEK 开始使用 AutoML 训练。

### 目标

在本教程中，您将学习如何在`Vertex AI AutoML`训练中使用客户管理的加密密钥（CMEK）。

本教程使用以下Google Cloud ML服务：

- `Vertex AI AutoML`
- 客户管理的加密密钥。

执行的步骤包括：

- 创建一个客户管理的加密密钥。
- 创建一个使用CMEK加密的图像数据集。
- 使用CMEK加密训练一个AutoML模型。

数据集

本教程使用的数据集是来自[TensorFlow](https://www.tensorflow.org/datasets/catalog/tf_flowers)的[Flowers数据集](https://www.tensorflow.org/datasets/catalog/overview)。本教程中的数据集版本存储在一个公共(GCS)存储桶中。训练的模型可以预测图像所属的花的类型，包括五种花：雏菊、蒲公英、玫瑰、向日葵或郁金香。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI
价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage
价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/)
基于您的预计使用情况生成成本估算。

## 安装

安装Vertex AI SDK和用于CMEK加密的KMS包。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade google-cloud-kms $USER_FLAG -q

重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，这样它才能找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

开始之前##

### 设置您的Google Cloud项目

**无论您使用的是笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用额度用于您的计算/存储成本。

1. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

1. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`前缀的行作为shell命令运行，并将以`$`前缀的Python变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改“REGION”变量，该变量用于本笔记本的其余操作。以下是Vertex AI支持的区域。我们建议您选择最靠近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能不能使用多区域存储桶进行Vertex AI训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在参加实时教程会话，您可能正在使用共享测试账户或项目。为避免用户在创建的资源上发生名称冲突，您可以为每个实例会话创建一个时间戳，并将时间戳附加到您在此教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的 Google 云账户

**如果您正在使用 Vertex AI Workbench 笔记本**，您的环境已经通过验证。请跳过此步骤。

如果您正在使用Colab，请运行下面的单元格，并按照提示进行oAuth身份验证。

否则，请按照以下步骤操作：

1. 在Cloud控制台中，转到[**创建服务帐号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐号**。

3. 在**服务帐号名称**字段中输入一个名称，然后点击**创建**。

4. 在**授予此服务帐号访问项目**部分，点击**角色**下拉列表。在过滤框中键入“Vertex AI”，并选择**Vertex AI管理员**。在过滤框中键入“存储对象管理员”，并选择**存储对象管理员**。

5. 点击*创建*。一个包含您密钥的JSON文件将下载到您的本地环境。

6. 在下面的单元格中将您的服务帐号密钥路径输入为`GOOGLE_APPLICATION_CREDENTIALS`变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，都需要按照以下步骤进行操作。**

当您初始化Python版的Vertex SDK时，您需要指定一个云存储分段桶。这个分段桶是您的数据集和模型资源相关数据在不同会话中保留的地方。

请在下方设置您的云存储桶的名称。桶的名称必须在所有Google Cloud项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶不存在时，才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

## 设置变量

接下来，设置一些在教程中使用的变量。
## 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import kms

### 初始化 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## 设置客户管理的加密密钥

默认情况下，Google Cloud 在数据存储在 Cloud Storage 中时会自动使用由 Google 管理的加密密钥对数据进行加密。如果您对保护数据的密钥有特定的合规性或监管要求，您可以为您的训练作业使用由客户管理的加密密钥（CMEK）。

### 启用 KMS API

首先，您需要启用[Cloud Key Management Service (KMS)](https://console.cloud.google.com/flows/enableapi?apiid=cloudkms.googleapis.com)

了解更多关于[客户管理的加密密钥（CMEK）](https://cloud.google.com/vertex-ai/docs/general/cmek)。

### 创建一个钥匙链

在启用了KMS API之后，您可以创建一个钥匙链和一把钥匙。 使用助手函数`create_key_ring()`来创建一个钥匙链，使用以下参数：

- `project_id`：您的项目ID。
- `location`：您的区域。
- `key_ring_id`：您的钥匙链的唯一标识符。

助手函数调用KMS客户端方法`create_key_ring()`来创建您的钥匙链。

了解更多关于[KMS：创建一个钥匙链](https://cloud.google.com/kms/docs/samples/kms-create-key-ring)。

In [ ]:
KEY_RING_ID = "your_cmek_key_ring_id"


def create_key_ring(project_id, location, key_ring_id):
    """
    Creates a new key ring in Cloud KMS

    Args:
        project_id (string): Google Cloud project ID (e.g. 'my-project').
        location (string): Cloud KMS location (e.g. 'us-east1').
        id (string): ID of the key ring to create (e.g. 'my-key-ring').

    Returns:
        KeyRing: Cloud KMS key ring.

    """

    # Create the client.
    client = kms.KeyManagementServiceClient()

    # Build the parent location name.
    location_name = f"projects/{project_id}/locations/{location}"

    # Build the key ring.
    key_ring = {}

    # Call the API.
    created_key_ring = client.create_key_ring(
        request={
            "parent": location_name,
            "key_ring_id": key_ring_id,
            "key_ring": key_ring,
        }
    )
    print("Created key ring: {}".format(created_key_ring.name))
    return created_key_ring


key_ring = create_key_ring(
    project_id=PROJECT_ID, location=REGION, key_ring_id=KEY_RING_ID
)
print(key_ring)

### 创建密钥

接下来，您需要创建您的密钥。使用帮助函数 `create_key()` 并传入以下参数：

- `project_id`：您的项目 ID。
- `location`：您的区域。
- `key_ring_id`：您的密钥环的唯一标识符。
- `key_id`：您的密钥的唯一标识符。

该帮助函数调用 KMS 客户端方法 `create_crypto_key()` 来创建您的密钥。

了解更多信息请访问 [](https://cloud.google.com/kms/docs/samples/kms-create-key-symmetric-encrypt-decrypt)

In [ ]:
KEY_ID = "your_cmek_key_id"


def create_key(project_id, location, key_ring_id, key_id):
    """
    Creates a new symmetric encryption/decryption key in Cloud KMS.

    Args:
        project_id (string): Google Cloud project ID (e.g. 'my-project').
        location (string): Cloud KMS location (e.g. 'us-east1').
        key_ring_id (string): ID of the Cloud KMS key ring (e.g. 'my-key-ring').
        key_id (string): ID of the key to create (e.g. 'my-symmetric-key').

    Returns:
        CryptoKey: Cloud KMS key.

    """

    # Create the client.
    client = kms.KeyManagementServiceClient()

    # Build the parent key ring name.
    key_ring_name = client.key_ring_path(project_id, location, key_ring_id)

    # Build the key.
    purpose = kms.CryptoKey.CryptoKeyPurpose.ENCRYPT_DECRYPT
    algorithm = (
        kms.CryptoKeyVersion.CryptoKeyVersionAlgorithm.GOOGLE_SYMMETRIC_ENCRYPTION
    )
    key = {
        "purpose": purpose,
        "version_template": {
            "algorithm": algorithm,
        },
    }

    # Call the API.
    created_key = client.create_crypto_key(
        request={"parent": key_ring_name, "crypto_key_id": key_id, "crypto_key": key}
    )
    print("Created symmetric key: {}".format(created_key.name))
    return created_key


key_id = create_key(
    project_id=PROJECT_ID, location=REGION, key_ring_id=KEY_RING_ID, key_id=KEY_ID
)

print(key_id)

### 设置服务账号权限

接下来，您需要为您的 Vertex AI 服务账号设置权限，以便使用您的密钥对资源进行加密和解密。

注意：此笔记本实例使用的计算引擎默认服务账号在进行 Google API 调用时用于身份验证，应被授予 Cloud KMS 管理员角色。

了解更多关于[授予 Vertex AI 权限](https://cloud.google.com/vertex-ai/docs/general/cmek#grant_permissions)。

In [ ]:
# Reference: https://cloud.google.com/vertex-ai/docs/general/cmek#granting_permissions
# Get the service account
SERVICE_ACCOUNT = ! gcloud projects get-iam-policy {PROJECT_ID} \
  --flatten="bindings[].members" \
  --format="table(bindings.members)" \
  --filter="bindings.role:roles/aiplatform.serviceAgent" \
  | grep -oP "service-.+?@gcp-sa-aiplatform.iam.gserviceaccount.com"
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]

print(f"Service account is: {SERVICE_ACCOUNT}")

# Give permissions
! gcloud kms keys add-iam-policy-binding {KEY_ID} \
  --keyring={KEY_RING_ID} \
  --location={REGION} \
  --project={PROJECT_ID} \
  --member=serviceAccount:{SERVICE_ACCOUNT} \
  --role=roles/cloudkms.cryptoKeyEncrypterDecrypter

为创建的密钥创建完整的资源标识符。

In [ ]:
ENCRYPTION_SPEC_KEY_NAME = key_id.name

## 初始化Python的Vertex SDK

为Vertex AI初始化*client*

在此笔记本运行期间创建的所有资源都将使用上面创建的加密密钥进行加密。

您可以在每个函数调用时覆盖加密密钥。

初始化Python项目的Vertex AI SDK

为您的项目、存储桶和对应的加密密钥初始化Vertex AI SDK。

本次会话中创建的所有资源都使用您创建的加密密钥进行加密。

*注意:* 您可以在每次函数调用时覆盖加密密钥。

In [ ]:
aiplatform.init(
    project=PROJECT_ID,
    staging_bucket=BUCKET_URI,
    location=REGION,
    encryption_spec_key_name=ENCRYPTION_SPEC_KEY_NAME,
)

#### 云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

# 创建 `Vertex AI ImageDataset` 资源

接下来，您将创建一个 `ImageDataset` 资源，并使用您的加密密钥对其进行加密。

In [ ]:
dataset = aiplatform.ImageDataset.create(
    display_name="flowers_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

发起一个训练作业以创建模型

训练一个AutoML图像分类模型。

In [ ]:
job = aiplatform.AutoMLImageTrainingJob(
    display_name="flowers_" + TIMESTAMP,
    prediction_type="classification",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)

# This will take around half an hour to run
model = job.run(
    dataset=dataset,
    model_display_name="flowers_" + TIMESTAMP,
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
)

部署您的模型，然后等待模型部署完成后再继续进行预测。

In [ ]:
endpoint = model.deploy()

# 在终端上进行预测
- 从导入到数据集中的数据中取一个样本
- 将该样本编码为base64并传递到终端进行预测。

In [ ]:
test_item = !gsutil cat $IMPORT_FILE | head -n1
test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

In [ ]:
import base64

import tensorflow as tf

with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()

# The format of each instance should conform to the deployed model's prediction input schema.
instances_list = [{"content": base64.b64encode(content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances_list)

print(prediction)

从端点中取消部署模型

In [ ]:
endpoint.undeploy_all()

清理

要清理此项目中使用的所有Google Cloud资源，您可以 [删除您在教程中使用的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源：

- 模型
- 数据集
- 云存储桶
- 终端节点

In [ ]:
# Delete endpoint resource
endpoint.delete()

# Delete model resource
model.delete()

# Delete dataset resource
dataset.delete()

# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

通过提供密钥版本值和其他参数来销毁CMEK。

In [ ]:
! gcloud kms keys versions destroy 1 \
  --key {KEY_ID} \
  --keyring={KEY_RING_ID} \
  --location={REGION} 

键列表

In [ ]:
! gcloud kms keys list --location {REGION} --keyring {KEY_RING_ID}